In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from fastai.tabular.all import *
import numpy as np

path = '/kaggle/input/tabular-playground-series-mar-2021/'

In [ ]:
df = pd.read_csv(path + '/train.csv')
df.head()

In [ ]:
df.describe(include='all')

In [ ]:
df.columns

In [ ]:
df.dtypes

In [ ]:
dls = TabularDataLoaders.from_csv(path + '/train.csv', path=path, y_names="target",
    cat_names = ['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7',
                 'cat8', 'cat9', 'cat10', 'cat11', 'cat12', 'cat13', 'cat14', 'cat15',
                 'cat16', 'cat17', 'cat18'],
    cont_names = ['cont0', 'cont1', 'cont2', 'cont3', 'cont4',
                  'cont5', 'cont6', 'cont7', 'cont8', 'cont9', 'cont10'],
    procs = [Categorify, FillMissing, Normalize])

In [ ]:
splits = RandomSplitter(valid_pct=0.2)(range_of(df))

In [ ]:
to_nn = TabularPandas(df, procs=[Categorify, FillMissing,Normalize],
                   cat_names = ['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7',
                                'cat8', 'cat9', 'cat10', 'cat11', 'cat12', 'cat13', 'cat14', 'cat15',
                                'cat16', 'cat17', 'cat18'],
    cont_names = ['cont0', 'cont1', 'cont2', 'cont3', 'cont4',
                  'cont5', 'cont6', 'cont7', 'cont8', 'cont9', 'cont10'],
                   y_names='target',
                   y_block = CategoryBlock,
                  splits=splits)

In [ ]:
dls = to_nn.dataloaders(bs=64)

In [ ]:
learn = tabular_learner(dls,layers=[500,200], 
                        metrics=[error_rate,
                                 accuracy])

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(2, 5e-2)

In [ ]:
learn.recorder.plot_loss()

In [ ]:
def r_mse(pred,y): return round(math.sqrt(((pred-y)**2).mean()), 6)
def m_rmse(m, xs, y): return r_mse(m.predict(xs), y)

In [ ]:
preds,targs = learn.get_preds()
r_mse(preds,targs)

In [ ]:
df_test = pd.read_csv(path + '/test.csv')

In [ ]:
test_df = df_test
# test_df.drop(['target'], axis=1, inplace=True)
dl = learn.dls.test_dl(test_df)

In [ ]:
preds_2 = learn.get_preds(dl=dl)

In [ ]:
arr = preds_2[0].numpy()

In [ ]:
df = pd.DataFrame(data=arr, 
                  index=pd.read_csv(path + '/test.csv')['id'], 
                  columns=["prob_0", "prob_1"]).reset_index()


def model_map(row):
    if row['prob_0'] >= 0.5:
        return 0
    else:
        return 1
    
df['Target'] = df.apply(lambda row : model_map(row), axis=1)

In [ ]:
df.drop(columns=['prob_0','prob_1'], inplace=True)

In [ ]:
df.head()

In [ ]:
df.to_csv('zw_submission.csv')